### Function Definitions

In [1]:
import json
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import classla
import json
import re

In [2]:
# input - name of a text file with one sentence per line
# output - list of sentences (strings)
def read_file_as_list_of_sentences(input_file_name):
    with open(input_file_name, "r") as input_file:
        return input_file.read().splitlines()

In [3]:
# input - list of sentences (strings) and a classla pipeline (POS tokenize or tokenize)
# output - list of dictionaries ((POS) tokenized sentences) 
def run_through_classla_pipeline(list_of_sentences, pipeline):
    return [pipeline(sentence).to_dict()[0][0] for sentence in list_of_sentences]

In [4]:
# input - list of dictionaries (a dictionary for each word - the tokenized version of the sentence)
# output - list of dictionaries (dictionaries that contain punctuation one after another are squashed into one)
def squash_punctuation(sentence):
    new_sentence = []

    for i in range(len(sentence)):
        if sentence[i]['text'] in ',()"[];.?!:-':
            if len(new_sentence) > 0 and all(character in ',()"[];.?!:-' for character in new_sentence[-1]['text']):
                new_sentence[-1]['text'] += sentence[i]['text']
            else:
                new_sentence.append(sentence[i])
        else:
            new_sentence.append(sentence[i])
            
    return new_sentence

In [5]:
# input - sentence and an index of a word (dictionary) - assigns a label to each word based on the punctuation after
# output - a label - None for punctuation, punctuation for words followed by punctuation and empty for words if they are not
def word2label(sentence, i):
    if all(character in ',()"[];.?!:-' for character in sentence[i]['text']):
        return None

    if i < len(sentence) - 1:
        if all(character in ',()"[];.?!:-' for character in sentence[i+1]['text']):
            label = sentence[i+1]['text']
            return label
    
    return ''

In [6]:
# input - sentence (list of dictionaries)
# output - list of labels (strings) with None labels for punctuation being filtered out
def sent2labels(sentence):
    return [label for label in (word2label(sentence, i) for i in range(len(sentence))) if label != None]

In [7]:
# input - list of dictionaries (a dictionary for each word - the tokenized version of the sentence)
# output - new sentence (string) with the punctuation removed
def remove_punctuation(sentence):
    new_sentence = ''

    for i in range(len(sentence)):
        if all(character in ',()"[];.?!:-' for character in sentence[i]['text']):
            pass
        else:
            new_sentence = new_sentence + sentence[i]['text'] + ' '
    
    return new_sentence

In [46]:
def contains_interrogative_word(sentence):
    for i in range(len(sentence)):
        if sentence[i]['xpos'] == 'Pi':
            return True, i
    
    return False, -1

In [47]:
def contains_interrogative_particle(sentence):
    for i in range(len(sentence)):
        if sentence[i]['xpos'] == 'Ti':
            return True, i
    
    return False, -1

In [48]:
# input - sentence (list of dictionaries) and an index of a word (dictionary)
# output - list of dictionaries (features, for each word)
def word2features(sentence, i):
    sentence_contains_interrogative_word, interrogative_word_position = contains_interrogative_word(sentence)
    sentence_contains_interrogative_particle, interrogative_particle_position = contains_interrogative_particle(sentence)

    features = {
        'word': sentence[i]['text'],
        'sent_len': len(sentence),
        'pos_in_sent': i,
        'upos': sentence[i]['upos'],
        'xpos': sentence[i]['xpos'],
        'first_word_in_sent': sentence[0]['text'],
        'contains_interrogative_word': sentence_contains_interrogative_word,
        'interrogative_word_position': interrogative_word_position,
        'contains_interrogative_particle': sentence_contains_interrogative_particle,
        'interrogative_particle_position': interrogative_particle_position
    }

    if i > 0:
        features.update({
            'prev_word': sentence[i-1]['text'],
            'prev_word_upos': sentence[i-1]['upos'],
            'prev_word_xpos': sentence[i-1]['xpos']
        })
    else:
        features.update({
            'BOS': True
        })

    if i < len(sentence)-1:
        features.update({
            'next_word': sentence[i+1]['text'],
            'next_word_upos': sentence[i+1]['upos'],
            'next_word_xpos': sentence[i+1]['xpos']
        })
    else:
        features.update({
            'EOS': True
        })
        
    if i < len(sentence)-2:
        features.update({
            'word_after_next_word': sentence [i+2]['text'],
            'word_after_next_word_upos': sentence[i+2]['upos'],
            'word_after_next_word_xpos': sentence[i+2]['xpos']
        })

    return features

In [9]:
# input - sentence (list of dictionaries)
# output - list of features (dictionaries, for each word)
def sent2features(sentence):
    return [word2features(sentence, i) for i in range(len(sentence))]

In [10]:
# input - JSON-serializable data and name of the output JSON file
# output - None, saves the sentences to a JSON file
def save_as_json(data, output_file_name):
    with open(output_file_name, "w") as output_file:
        json.dump(data, output_file) 

In [11]:
# input - JSON file
# output - the contents of the JSON file as an object
def load_json(json_file_name):
    with open(json_file_name, "r") as json_file:
        return json.load(json_file)

In [12]:
# input - name of a text file with one sentence per line and a variable indicating whether or not to save X and y to JSON
# output - X and y - features and labels
def data_prep(input_file_name, json_serialize=False):
    data = read_file_as_list_of_sentences(input_file_name)
    nlp_tokenize = classla.Pipeline('bg', processors='tokenize')
    tokenized_data = run_through_classla_pipeline(data, nlp_tokenize)
    
    if len(data) != len(tokenized_data):
        print("Warning: Mismatch in the count of the data and tokenized data")

    squashed_tokenized_data = [squash_punctuation(sentence) for sentence in tokenized_data]

    if len(tokenized_data) != len(squashed_tokenized_data):
        print("Warning: Mismatch in the count of the tokenized and squashed tokenized data")
    
    y = [sent2labels(sentence) for sentence in squashed_tokenized_data]
    
    if len(squashed_tokenized_data) != len(y):
        print("Warning: Mismatch in the count of the squashed tokenized data and labeled data")
    
    data_without_punctuation = [remove_punctuation(sentence) for sentence in squashed_tokenized_data]
    
    if len(data_without_punctuation) != len(y):
        print("Warning: Mismatch in the count of the data without punctuation and labeled data")
    
    nlp_pos_tokenize = classla.Pipeline('bg', processors='tokenize,pos')   
    pos_tokenized_data = run_through_classla_pipeline(data_without_punctuation, nlp_pos_tokenize)
    
    if len(data_without_punctuation) != len(pos_tokenized_data):
        print("Warning: Mismatch in the count of the data without punctuation and POS tokenized data")
    
    X = [sent2features(sentence) for sentence in pos_tokenized_data]
    
    if len(X) != len(pos_tokenized_data):
        print("Warning: Mismatch in the count of the prepped data and POS tokenized data")
    
    if json_serialize:
        save_as_json(X, re.sub('\.txt', '_X.json', input_file_name))
        save_as_json(y, re.sub('\.txt', '_y.json', input_file_name))
        
    return X, y

In [13]:
# input - X and y - features and labels
# output - None, prints on the screen the features-labels pairs that have length mismatch
def verify_prepped_data(X, y):
    for feat, label in zip(X, y):
        if len(feat) != len(label):
            print(feat, label)

In [14]:
# input - X and y - features and labels
# output - list of punctuated sentences (strings; y labels applied to X)
def punctuate(X, y):
    punctuated_sentences = []

    for feat, label in zip(X, y):
        sentence = ''

        for i in range(len(feat)):
            sentence = sentence + feat[i]['word'] + label[i] + ' '
        
        punctuated_sentences.append(sentence.rstrip())
    
    return punctuated_sentences

### Data Prep

In [49]:
%%time
X_train, y_train = data_prep('../data/Train.txt')

2021-09-27 15:18:13 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2021-09-27 15:18:13 INFO: Use device: gpu
2021-09-27 15:18:13 INFO: Loading: tokenize
2021-09-27 15:18:13 INFO: Done loading processors!
2021-09-27 15:18:39 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |

2021-09-27 15:18:39 INFO: Use device: gpu
2021-09-27 15:18:39 INFO: Loading: tokenize
2021-09-27 15:18:39 INFO: Loading: pos
2021-09-27 15:18:41 INFO: Done loading processors!


CPU times: user 22min 59s, sys: 2.25 s, total: 23min 1s
Wall time: 23min


In [50]:
verify_prepped_data(X_train, y_train)

In [18]:
X_train_loaded_from_json = load_json("../data/Train_X.json")
y_train_loaded_from_json = load_json("../data/Train_y.json")

In [20]:
verify_prepped_data(X_train_loaded_from_json, y_train_loaded_from_json)

In [52]:
%%time
X_dev, y_dev = data_prep('../data/Dev.txt')

2021-09-27 16:14:17 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2021-09-27 16:14:17 INFO: Use device: gpu
2021-09-27 16:14:17 INFO: Loading: tokenize
2021-09-27 16:14:17 INFO: Done loading processors!
2021-09-27 16:14:25 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |

2021-09-27 16:14:25 INFO: Use device: gpu
2021-09-27 16:14:25 INFO: Loading: tokenize
2021-09-27 16:14:25 INFO: Loading: pos
2021-09-27 16:14:26 INFO: Done loading processors!


CPU times: user 11min 34s, sys: 1.05 s, total: 11min 35s
Wall time: 11min 34s


In [42]:
verify_prepped_data(X_dev, y_dev)

### CRF Model

In [51]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 30min 15s, sys: 1.64 s, total: 30min 16s
Wall time: 30min 13s


In [53]:
labels = list(crf.classes_)

In [54]:
labels

['',
 ':',
 '!',
 ',',
 '?',
 ';',
 '.',
 '-',
 ',(',
 '"',
 '").',
 '(',
 ')',
 '?-',
 '.-',
 ':)',
 '!-',
 ':"',
 '."',
 ':(',
 '?)',
 '"?',
 '".',
 '";',
 '"-',
 ').',
 '!,',
 '-"',
 ')!',
 '",',
 '.")',
 '!;',
 ';(',
 '),',
 '?;',
 '?"',
 ');',
 '.)',
 '?,',
 ';"',
 '):',
 '[',
 ']',
 '...',
 '],',
 '"(',
 ',[',
 ',"',
 ']",',
 ')-',
 '":',
 '("',
 '"),',
 '"-[',
 '...,',
 '")',
 '..',
 '","',
 '?".',
 '"[',
 '"("',
 '),"',
 '""',
 '-.',
 '.,',
 '].',
 ')"',
 '!"',
 '?"-',
 '":"',
 '):"',
 '")-',
 '"-"',
 '-,',
 ')".',
 '?",',
 ']".',
 '!",',
 ']?',
 '...[',
 '];',
 '...".',
 '!"-',
 '"!',
 '!".',
 '!?',
 '?!',
 ',-',
 '!...',
 '!).',
 ')?',
 '?...',
 '.?']

In [55]:
y_pred = crf.predict(X_dev)

In [56]:
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.9364657048718186

In [57]:
# show all metrics
# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=labels, digits=3
))

/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['', ':', '!', ',', '?', ';', '.', '-', ',(', '"', '").', '(', ')', '?-', '.-', ':)', '!-', ':"', '."', ':(', '?)', '"?', '".', '";', '"-', ').', '!,', '-"', ')!', '",', '.")', '!;', ';(', '),', '?;', '?"', ');', '.)', '?,', ';"', '):', '[', ']', '...', '],', '"(', ',[', ',"', ']",', ')-', '":', '("', '"),', '"-[', '...,', '")', '..', '","', '?".', '"[', '"("', '),"', '""', '-.', '.,', '].', ')"', '!"', '?"-', '":"', '):"', '")-', '"-"', '-,', ')".', '?",', ']".', '!",', ']?', '...[', '];', '...".', '!"-', '"!', '!".', '!?', '?!', ',-', '!...', '!).', ')?', '?...', '.?'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0

              precision    recall  f1-score   support

                  0.958     0.989     0.973    227825
           :      0.901     0.650     0.755       923
           !      0.505     0.161     0.245       347
           ,      0.793     0.635     0.706     18592
           ?      0.875     0.506     0.641       443
           ;      0.286     0.048     0.083       622
           .      0.916     0.985     0.949     11401
           -      0.550     0.087     0.151       882
          ,(      0.000     0.000     0.000         5
           "      0.603     0.128     0.212      1596
         ").      0.000     0.000     0.000         0
           (      0.966     0.871     0.916      1656
           )      0.762     0.763     0.763      1001
          ?-      0.000     0.000     0.000         4
          .-      0.000     0.000     0.000         2
          :)      0.000     0.000     0.000         0
          !-      0.000     0.000     0.000        20
          :"      0.667    

In [58]:
labels.remove('')

In [59]:
y_pred = crf.predict(X_dev)

In [60]:
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.7247699161656445

In [61]:
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=labels, digits=3
))

/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[':', '!', ',', '?', ';', '.', '-', ',(', '"', '").', '(', ')', '?-', '.-', ':)', '!-', ':"', '."', ':(', '?)', '"?', '".', '";', '"-', ').', '!,', '-"', ')!', '",', '.")', '!;', ';(', '),', '?;', '?"', ');', '.)', '?,', ';"', '):', '[', ']', '...', '],', '"(', ',[', ',"', ']",', ')-', '":', '("', '"),', '"-[', '...,', '")', '..', '","', '?".', '"[', '"("', '),"', '""', '-.', '.,', '].', ')"', '!"', '?"-', '":"', '):"', '")-', '"-"', '-,', ')".', '?",', ']".', '!",', ']?', '...[', '];', '...".', '!"-', '"!', '!".', '!?', '?!', ',-', '!...', '!).', ')?', '?...', '.?'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/mariya/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 i

              precision    recall  f1-score   support

           :      0.901     0.650     0.755       923
           !      0.505     0.161     0.245       347
           ,      0.793     0.635     0.706     18592
           ?      0.875     0.506     0.641       443
           ;      0.286     0.048     0.083       622
           .      0.916     0.985     0.949     11401
           -      0.550     0.087     0.151       882
          ,(      0.000     0.000     0.000         5
           "      0.603     0.128     0.212      1596
         ").      0.000     0.000     0.000         0
           (      0.966     0.871     0.916      1656
           )      0.762     0.763     0.763      1001
          ?-      0.000     0.000     0.000         4
          .-      0.000     0.000     0.000         2
          :)      0.000     0.000     0.000         0
          !-      0.000     0.000     0.000        20
          :"      0.667     0.060     0.110        67
          ."      0.200    

In [37]:
save_as_json(X_train, "../data/Тrain_X.json")

In [3]:
X_dev_loaded_from_json = load_json("../data/Dev_X.json")

In [4]:
X_dev_loaded_from_json[0]

[{'word': 'Но',
  'sent_len': 41,
  'pos_in_sent': 0,
  'upos': 'CCONJ',
  'xpos': 'Cc',
  'first_word_in_sent': 'Но',
  'BOS': True,
  'next_word': 'синовете'},
 {'word': 'синовете',
  'sent_len': 41,
  'pos_in_sent': 1,
  'upos': 'NOUN',
  'xpos': 'Ncmpd',
  'first_word_in_sent': 'Но',
  'prev_word': 'Но',
  'next_word': 'им'},
 {'word': 'им',
  'sent_len': 41,
  'pos_in_sent': 2,
  'upos': 'PRON',
  'xpos': 'Pszt--3',
  'first_word_in_sent': 'Но',
  'prev_word': 'синовете',
  'next_word': 'не'},
 {'word': 'не',
  'sent_len': 41,
  'pos_in_sent': 3,
  'upos': 'PART',
  'xpos': 'Tn',
  'first_word_in_sent': 'Но',
  'prev_word': 'им',
  'next_word': 'умъртви'},
 {'word': 'умъртви',
  'sent_len': 41,
  'pos_in_sent': 4,
  'upos': 'VERB',
  'xpos': 'Vpptf-o3s',
  'first_word_in_sent': 'Но',
  'prev_word': 'не',
  'next_word': 'защото'},
 {'word': 'защото',
  'sent_len': 41,
  'pos_in_sent': 5,
  'upos': 'ADV',
  'xpos': 'Prc',
  'first_word_in_sent': 'Но',
  'prev_word': 'умъртви',
  'ne

In [38]:
save_as_json(y_train, "../data/Тrain_y.json")

In [41]:
y_dev_loaded_from_json = load_json("../data/Dev_y.json")

In [40]:
verify_prepped_data(X_dev, X_dev_loaded_from_json)

In [42]:
verify_prepped_data(y_dev, y_dev_loaded_from_json)

In [37]:
save_as_json(X_dev, "../data/Dev_X.json")

In [38]:
save_as_json(y_dev, "../data/Dev_y.json")

In [62]:
punctuated_sentences = punctuate(X_dev, y_pred)

In [63]:
punctuated_sentences[:100]

['Но синовете им не умъртви, защото постъпи според писаното в закона в книгата на Мойсей, където ГОСПОД заповяда и каза: Бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях.',
 'Но когато ти говоря ще отворя устата ти и ти им кажи: Така казва Господ БОГ: Който слуша, нека слуша, а който не слуша нека не слуша, защото са бунтовен дом.',
 'Защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува?',
 'А след това ще се насели, както в предишните дни, заявява ГОСПОД.',
 'А Той беше на задната част, заспал на възглавница.',
 'Ефрем е като птица славата му ще отлети.',
 'И така той си отиде от него на известно разстояние.',
 'Не бягам при халдейците!',
 'И израилевите синове излязоха от египетската земя строени.',
 'А когато Ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си работи и се обеси.',
 'Ако умре някое животно от добитъка, който 

In [48]:
punctuated_sentences[:100]

['Но синовете им не умъртви, защото постъпи според писаното в закона в книгата на Мойсей, където ГОСПОД заповяда и каза: Бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях.',
 'Но когато ти говоря ще отворя устата ти и ти им кажи: Така казва Господ БОГ: Който слуша нека слуша, а който не слуша нека не слуша, защото са бунтовен дом.',
 'Защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува.',
 'А след това ще се насели, както в предишните дни, заявява ГОСПОД.',
 'А Той беше на задната част заспал на възглавница.',
 'Ефрем е като птица славата му ще отлети.',
 'И така той си отиде от него на известно разстояние.',
 'Не бягам при халдейците!',
 'И израилевите синове излязоха от египетската земя строени.',
 'А когато Ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си работи и се обеси.',
 'Ако умре някое животно от добитъка, който мо

In [49]:
correct_punctuated_sentences = punctuate(X_dev, y_dev)

In [50]:
correct_punctuated_sentences[:100]

['Но синовете им не умъртви, защото постъпи, според писаното в закона, в книгата на Мойсей, където ГОСПОД заповяда и каза: Бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях.',
 'Но когато ти говоря, ще отворя устата ти и ти им кажи: Така казва Господ БОГ: Който слуша, нека слуша, а който не слуша, нека не слуша- защото са бунтовен дом.',
 'Защото кой е по-голям: този, който седи на трапезата ли, или онзи, който слугува?',
 'А след това ще се насели както в предишните дни, заявява ГОСПОД.',
 'А Той беше на задната част, заспал на възглавница.',
 'Ефрем е като птица, славата му ще отлети.',
 'И така, той си отиде от него на известно разстояние.',
 'Не бягам при халдейците.',
 'И израилевите синове излязоха от египетската земя строени.',
 'А когато Ахитофел видя, че съветът му не се изпълни, оседла магарето си и стана, и отиде у дома си, в своя град, и нареди домашните си работи, и се обеси.',
 'Ако умре някое животно от доби

### Grid Search Try - Worse Results

In [ ]:
# from sklearn.metrics import make_scorer
# import scipy.stats
# from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# %%time
# # define fixed parameters and parameters to search
# crf = sklearn_crfsuite.CRF(
#     algorithm='lbfgs',
#     max_iterations=100,
#     all_possible_transitions=True
# )
# params_space = {
#     'c1': scipy.stats.expon(scale=0.5),
#     'c2': scipy.stats.expon(scale=0.05),
# }

# # use the same metric for evaluation
# f1_scorer = make_scorer(metrics.flat_f1_score,
#                         average='weighted', labels=labels)

# # search
# rs = RandomizedSearchCV(crf, params_space,
#                         cv=3,
#                         verbose=1,
#                         n_jobs=-1,
#                         n_iter=50,
#                         scoring=f1_scorer)
# rs.fit(X, y)

In [ ]:
# crf = rs.best_estimator_
# print('best params:', rs.best_params_)
# print('best CV score:', rs.best_score_)
# print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
# crf = rs.best_estimator_
# y_pred = crf.predict(X_test)

# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )

# print(metrics.flat_classification_report(
#     y_test, y_pred, labels=sorted_labels, digits=3
# ))